In [1]:
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# Simulation parameters
N         = 100   # Number of particles 
t         = 0      # current time of the simulation
tEnd      = 5.0   # time at which simulation ends 
dt        = 0.01   # timestep 
soft      = 0.1    # soft length
pi        = np.pi  # pi
G         = 1.0   # Newton's Gravitational Constant in AU
plotRealTime = True # switch on for plotting as the simulation goes along
    
# Generate Initial Conditions
np.random.seed(20)            # set the random number generator seed

In [3]:
def unitary_sphere(N):
    pos = np.zeros([N,3])
    for k in range(N):
        r = 1.0 * np.random.rand() # 0<r<1
        theta = pi*np.random.rand() #0<theta<2*pi
        phi = 2*pi*np.random.rand() #0<phi<pi
        pos[k][0] = r * np.sin(theta) * np.cos(phi)
        pos[k][1] = r * np.sin(theta) * np.sin(phi)
        pos[k][2] = r * np.cos(theta)
    return pos

In [4]:
mass0 = 10.0 * np.random.rand(N,1)
mass0 = sorted(mass0)
   
pos = unitary_sphere(N)  # Randomly selected positions and velocities
vel = np.zeros([N,3])

In [5]:
mass = np.zeros([N,1])
for i in range(0,N):
    mass[i]=mass0[i]   

In [6]:
def Aceleracion( pos, mass, G, soft ):
    # Positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]
    
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # Matrix for all particles  
    inv = (dx**2 + dy**2 + dz**2 + soft**2)
    inv[inv>0] = inv[inv>0]**(-3/2)
    
    # Acceleration a=[x,y,z] for all particles
    ax = (G*dx*inv) @mass
    ay = (G*dy*inv) @mass
    az = (G*dz*inv) @mass
    
    # Pack together the acceleration components
    a = np.hstack((ax,ay,az))
    return a

In [7]:
def Energia( pos, vel, mass, G ):
    # Kinetic Energy:
    KinE = (1/2) * np.sum(np.sum(mass * vel**2))
    
    # Potential Energy:
    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]
    
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # Matrix for all particles 1/r
    inv_r = np.sqrt(dx**2 + dy**2 + dz**2)
    inv_r[inv_r>0] = 1.0/inv_r[inv_r>0]
    
    # Sum over upPotEr triangle, to count each interaction only once
    PotE = G * np.sum(np.sum(np.triu(-(mass*mass.T)*inv_r,1)))
    
    return KinE, PotE;

In [8]:
%matplotlib

# calculate initial gravitational accelerations
acc = Aceleracion( pos, mass, G, soft )

# calculate initial energy of system
KinE, PotE  = Energia( pos, vel, mass, G )

# number of timesteps
Nt = int(tEnd/dt)

# save energies, particle orbits for plotting trails
final_pos = np.zeros((N,3,Nt+1))
final_pos[:,:,0] = pos
Final_KinE = np.zeros(Nt+1)
Final_KinE[0] = KinE
Final_PotE = np.zeros(Nt+1)
Final_PotE[0] = PotE
time = np.arange(Nt+1)*dt

# prep figure
fig = plt.figure(figsize=(10,10), dpi=80)
grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
ax1 = plt.subplot(grid[0:2,0])
ax2 = plt.subplot(grid[2,0])

# Simulation Main Loop
for i in range(Nt):
    # (1/2) kick
    vel = vel+((acc*dt)/2)
    
    # drift
    pos =pos+(vel*dt)
    
    # update accelerations
    acc = Aceleracion( pos, mass, G, soft )
    
    # (1/2) kick
    vel =vel+((acc*dt)/2)
    
    # update time
    t = t+dt
    
    # Get energy of system
    KinE, PotE  = Energia( pos, vel, mass, G )
    
    # Final positions and energies 
    final_pos[:,:,i+1] = pos
    Final_KinE[i+1] = KinE
    Final_PotE[i+1] = PotE
    
    # Plot 
    if plotRealTime or (i == Nt-1):
        plt.sca(ax1)
        plt.cla()
        xx = final_pos[:,0,max(i-50,0):i+1]
        yy = final_pos[:,1,max(i-50,0):i+1]
        plt.scatter(xx,yy,s=1,color=[.7,.7,1])
        plt.scatter(pos[0:50,0],pos[0:50,1],s=10,color='red')
        plt.scatter(pos[51:99,0],pos[51:99,1],s=10,color='black')
        ax1.set(xlim=(-10, 10), ylim=(-10, 10))
        ax1.set_aspect('equal', 'box')
        
        plt.sca(ax2)
        plt.cla()
        plt.scatter(time,Final_KinE,color='red',s=1,label='Kinetic Energy' if i == Nt-1 else "")
        plt.scatter(time,Final_PotE,color='blue',s=1,label='Potential Energy' if i == Nt-1 else "")
        plt.scatter(time,Final_KinE+Final_PotE,color='black',s=1,label='Total Energy' if i == Nt-1 else "")
        plt.scatter(time,Final_PotE/2.0,color='orange',s=1,label='Virial' if i == Nt-1 else "")
        plt.xlabel('Time')
        plt.ylabel('Energy')
        ax2.set(xlim=(0, tEnd), ylim=(-200000, 200000))
        ax2.set_aspect('auto', 'box')        
        plt.pause(0.0001)
    
plt.sca(ax2)
plt.xlabel('time')
plt.ylabel('energy')
ax2.legend(loc='upper right')
plt.grid()
plt.show()

Using matplotlib backend: Qt5Agg


## Esfera Unitaria 

In [ ]:
%matplotlib inline
fig2 = plt.figure(figsize=(6,8), dpi=80)
grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
theta = np.linspace(0, 2*np.pi, 200)
xc =np.cos(theta)
yc =np.sin(theta)

ax1 = plt.subplot(grid[0:2,0])
plt.scatter(final_pos[0:50,0,1],final_pos[0:50,1,1],s=10,color='red')
plt.scatter(final_pos[51:99,0,1],final_pos[51:99,1,1],s=10,color='black')
plt.scatter(xc,yc,s=1,color='blue')
plt.xlabel('AU')
plt.ylabel('AU')
ax1.set(xlim=(-1.1, 1.1), ylim=(-1.1, 1.1))
ax1.set_aspect('equal', 'box')
plt.show()
#plt.savefig("dist_inic.png",bbox_inches='tight',dpi=400) #guarda en .png

## Perfil de Masas

In [21]:
tiempo=100 #<<---------------instante de tiempo en multiplos de dt 0-1-3-10-30-100
nn=1000 #<<--------num de puntos en el array R (no cambia mucho al aumentarlo, puesto que son solo 100 masas)
xlimit=20 #<<--------------- Radio maximo donde quiero visualizar
R = np.linspace(0,xlimit,nn)
M = np.zeros(nn)

for l in range(0,nn):
    mass2 = np.zeros(100)
    for i in range(0,100):
        norma = np.sqrt(final_pos[i,0,tiempo]**2 + final_pos[i,1,tiempo]**2 + final_pos[i,2,tiempo]**2)
        if norma>R[l]:
            mass2[i]=mass[i]*1.0
        else:
            mass2[i]=mass[i]*0.0
    M[l]=sum(mass2)    

In [22]:
fig3 = plt.figure(figsize=(6,6), dpi=80)
plt.plot(R,M,'-r')
plt.xlabel('AU')
plt.ylabel('$M_\odot$')
plt.show()
plt.savefig("perfil_masa.png",bbox_inches='tight',dpi=400) #guarda en .png